#  RAG + Recursive Editing with Phi-3-mini

This notebook demonstrates a simple **Retrieval-Augmented Generation (RAG)** pipeline combined with **Recursive Editing** using Microsoft's small language model **Phi-3-mini (4K instruct)**.

## Step 1: Setup and Installation

We install all required libraries for running the Phi-3-mini model and RAG pipeline:

- **API connection** (`huggingface_hub`)  
- **RAG and indexing** (`llama-index-core`, `llama-index-embeddings-huggingface`)  
- **Text embeddings** (`sentence-transformers`)  
- **PDF reading** (`pypdf`)  
- **PyTorch backend** (`torch`)  
- **Energy tracking** (`codecarbon`)  
- **TensorFlow/Keras backend support** (`tf-keras`)


In [1]:
!pip install huggingface_hub
!pip install llama-index-core
!pip install llama-index-embeddings-huggingface
!pip install sentence-transformers
!pip install pypdf
!pip install torch
!pip install codecarbon
!pip install tf-keras


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 8.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━

##  Step 2: Import Libraries
We import all libraries and configure:

- **Environment variables**  
- **API key**  
- **Path to Apollo dataset**  
- **Country code** for emissions tracking

Make sure to update `DATA_PATH` with the location of your Apollo documents.


In [14]:
import time
import textwrap
from huggingface_hub import InferenceClient
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.embeddings import resolve_embed_model
from codecarbon import OfflineEmissionsTracker
from dotenv import load_dotenv
import os
from llama_index.core.embeddings import resolve_embed_model


# Load API key and model from .env
load_dotenv()
HF_API_KEY = os.getenv("HF_TOKEN")
HF_MODEL_ID = os.getenv("HF_MODEL")

# this points to your the Apollo dataset
DATA_PATH = r"phi3_model/data"

YOUR_COUNTRY_ISO_CODE = "EGY"

if not HF_API_KEY or not HF_MODEL_ID:
    raise ValueError(" HF_TOKEN or HF_MODEL not found. Check your .env file!")

print(f" Loaded HF API key and model: {HF_MODEL_ID}")
print(f" Data path: {DATA_PATH}")

 Loaded HF API key and model: microsoft/phi-3-mini-4k-instruct
 Data path: phi3_model/data


## Step 3 — Define Prompts

We define three prompt templates for our recursive refinement loop:

1. **Draft Prompt** — initial answer generation.  
2. **Critic Prompt** — evaluates draft for accuracy and relevance.  
3. **Refiner Prompt** — rewrites the draft based on critic feedback.

This allows for **structured, iterative improvement** of answers.


In [15]:
DRAFT_SYSTEM_PROMPT = (
    "You are an expert assistant. Answer the user's question based *only* on the provided context. "
    "Do not mention the context or 'the text'. If the answer is not in the context, say so."
)
DRAFT_USER_TEMPLATE = "Context:\n{context_str}\n\nQuestion:\n{query_str}"

CRITIC_SYSTEM_PROMPT = (
    "You are a 'Critic' AI. Evaluate the 'Draft Answer' based *only* on the 'Source Context'. "
    "Find flaws in Faithfulness (is it supported by context?) and Relevance (does it answer the question?). "
    "Provide feedback as bullet points. If perfect, respond with 'The draft is perfect.'"
)
CRITIC_USER_TEMPLATE = "Source Context:\n{context}\n\nOriginal Question:\n{question}\n\nDraft Answer:\n{draft}"

REFINER_SYSTEM_PROMPT = (
    "You are a 'Refiner' AI. Rewrite the 'Original Draft' to incorporate the 'Critic's Feedback'. "
    "Do not add new info. Do not mention the context, draft, or feedback. Just provide the final, improved answer."
)
REFINER_USER_TEMPLATE = "Original Draft:\n{draft}\n\nCritic's Feedback:\n{feedback}"

print(" Prompts defined!")

 Prompts defined!


## Step 4 — Initialize API and Index Data

- Connect to the Hugging Face API.  
- Create an **embedding model** for retrieval.  
- Load Apollo documents and build a **vector index**.  
- Prepare a **retriever** to fetch the most relevant document chunks.


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.embeddings import resolve_embed_model
import os

DATA_PATH = r"C:\Users\DELL\ELO2_GREEN_AI\ELO2_GREEN_AI\phi3_model\data"

# check folder
print("Folder exists?", os.path.exists(DATA_PATH))
print("Folder contents:", os.listdir(DATA_PATH))

# Embeddings for retrieval
embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# Load documents
documents = SimpleDirectoryReader(DATA_PATH).load_data()

# Build vector index
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
retriever = index.as_retriever(similarity_top_k=3)

print(" Retriever is ready.")

Folder exists? True
Folder contents: ['source.txt']
✅ Retriever is ready.


## Step 5 — Helper Function to Call Hugging Face API

This function sends prompts to the API and returns the response.  

- Includes **retry logic** in case of temporary API failure.  
- Ensures that **all API calls are consistent** for draft, critic, and refiner steps.


In [ ]:
def call_hf_api(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    retries = 3
    for attempt in range(retries):
        try:
            # create HF Inference client (put this after load_dotenv() and HF_API_KEY is set)
            from huggingface_hub import (
                InferenceClient,
            )  # if not already imported in an earlier cell

            client = InferenceClient(token=HF_API_KEY)
            return response_full.choices[0].message.content.strip()
        except Exception as e:
            print(f"API Error (Attempt {attempt + 1}/{retries}): {e}")
            if attempt < retries - 1:
                time.sleep(10)
    return "Error: API failed."

## Step 6 — Helper Function to Call Hugging Face API

This function sends prompts to the API and returns the response.  

- Includes **retry logic** in case of temporary API failure.  
- Ensures that **all API calls are consistent** for draft, critic, and refiner steps.


In [22]:
def call_hf_api(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    retries = 3
    for attempt in range(retries):
        try:
            response_full = client.chat_completion(
                messages=messages,
                model=HF_MODEL_ID,
                max_tokens=1024,
                temperature=0.1,
            )
            return response_full.choices[0].message.content.strip()
        except Exception as e:
            print(f"API Error (Attempt {attempt + 1}/{retries}): {e}")
            if attempt < retries - 1:
                time.sleep(10)
    return "Error: API failed."

## Step 7 — Recursive RAG Loop with CodeCarbon

- Ask a question about Apollo.  
- Retrieve relevant context using the retriever.  
- Generate an initial **draft answer**.  
- Run **refinement cycles**:
  - Critic evaluates draft
  - Refiner updates draft based on feedback
- Track **CPU energy and CO2 emissions** for each query.  
- Print final refined answer with **time and emission metrics**.


In [ ]:
REFINEMENT_CYCLES = 3
tracker = OfflineEmissionsTracker(country_iso_code=YOUR_COUNTRY_ISO_CODE)
tracker.start()

print(f"\n--- Query Engine Ready (API-Powered, {REFINEMENT_CYCLES} cycles) ---")
print("Type 'exit' to quit.\n")

try:
    while True:
        # Replace the query to run a loop against the prompts of the pre-determined questions.
        query = input("Ask a question about Apollo: ")
        if query.lower() == "exit":
            break

        tracker.start_task("Recursive RAG Query")
        start_time = time.time()

        retrieved_nodes = retriever.retrieve(query)
        context_str = "\n---\n".join([node.get_content() for node in retrieved_nodes])

        user_prompt = DRAFT_USER_TEMPLATE.format(
            context_str=context_str, query_str=query
        )
        draft_text = call_hf_api(DRAFT_SYSTEM_PROMPT, user_prompt)
        print("\n📝 Initial Draft:\n", textwrap.fill(draft_text, width=80))

        current_draft = draft_text

        for i in range(REFINEMENT_CYCLES):
            print(f"\n🔁 Refinement Cycle {i + 1}/{REFINEMENT_CYCLES}")

            critic_prompt = CRITIC_USER_TEMPLATE.format(
                context=context_str, question=query, draft=current_draft
            )
            feedback = call_hf_api(CRITIC_SYSTEM_PROMPT, critic_prompt)
            print("\n🧩 Critic Feedback:\n", feedback)

            if "The draft is perfect" in feedback:
                print("✅ Draft approved early!")
                break

            refiner_prompt = REFINER_USER_TEMPLATE.format(
                draft=current_draft, feedback=feedback
            )
            current_draft = call_hf_api(REFINER_SYSTEM_PROMPT, refiner_prompt)
            print("\n✏️ Refined Draft:\n", textwrap.fill(current_draft, width=80))

        final_answer = current_draft
        end_time = time.time()
        emissions_data = tracker.stop_task()

        print("\n✅ Final Answer:\n", textwrap.fill(final_answer, width=80))
        print(f"\n--- Query Metrics ---\nTime: {end_time - start_time:.2f}s")
        print(f"Emissions: {emissions_data.emissions * 1000:.6f} gCO2eq")

finally:
    total_emissions = tracker.stop()
    print("\n🌍 Total Emissions Summary:")
    print(f"Total Local CO2: {total_emissions * 1000:.6f} gCO2eq")
    print("Report saved as 'emissions.csv'")

[codecarbon INFO @ 05:41:59] offline tracker init
[codecarbon WARNING @ 05:41:59] Multiple instances of codecarbon are allowed to run at the same time.



--- Query Engine Ready (API-Powered, 3 cycles) ---
Type 'exit' to quit.

API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

📝 Initial Draft:
 Error: API failed.

🔁 Refinement Cycle 1/3
API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

🧩 Critic Feedback:
 Error: API failed.
API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

✏️ Refined Draft:
 Error: API failed.

🔁 Refinement Cycle 2/3
API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

🧩 Critic Feedback:
 Error: API failed.
API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

✏️ Refined Draft:
 Error: API failed.

🔁 Refinement Cycle 3/3
API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

🧩 Critic Feedback:
 Error: API failed.
API Error (Attempt 1/3): 
API Error (Attempt 2/3): 
API Error (Attempt 3/3): 

✏️ Refined Draft:
 Error: API failed.

✅ Final Answer:
 Error: API fail

c:\Users\DELL\AppData\Local\Programs\Python\Python313\Lib\site-packages\codecarbon\output_methods\file.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)



🌍 Total Emissions Summary:
Total Local CO2: 13.644425 gCO2eq
Report saved as 'emissions.csv'


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'